In [1]:
import numpy as np
import csv
import pandas as pd
import os
import time
from tqdm import tqdm
from multiprocessing import Pool

In [2]:
df_ques = pd.read_pickle('/Users/s0c02nj/Downloads/MS_marco_subset/df_query_ms_500.pkl')

In [3]:
df_res = pd.read_pickle('/Users/s0c02nj/Downloads/MS_marco_subset/df_result_ms_500.pkl')

In [4]:
from elasticsearch import helpers, Elasticsearch
import csv
import re
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from inflection import singularize
import matplotlib.pyplot as plt
from scipy.spatial.distance import cosine as cs

In [5]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
es.info()

{'name': 'm-c02wv1jnhtd5',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'i0Q_KCKeTLGlRZzF8qt__A',
 'version': {'number': '7.6.2',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': 'ef48eb35cf30adf4db14086e8aabd07ef6fb113f',
  'build_date': '2020-03-26T06:34:37.794943Z',
  'build_snapshot': False,
  'lucene_version': '8.4.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [6]:
def text_preprocessing(text):
    text= text.lower()
    text= re.sub(r'[^a-z]',' ',text)
    #text= " ".join([s for s in text.split() if len(s)>2])
    #text= " ".join([x for x in text.split() if x not in stopwords.words('english')])
    #text= " ".join([inflection.singularize(x) for x in text.split()])
    text= text.strip()
    return(text)

In [7]:
tqdm.pandas()

In [8]:
df_ques['passage'] = df_ques['passage'].progress_apply(lambda x: text_preprocessing(x))

df_ques['query'] = df_ques['query'].progress_apply(lambda x: text_preprocessing(x))

100%|██████████| 497308/497308 [00:02<00:00, 222430.36it/s]


In [9]:
index='index_k1'

In [10]:
def doc_generator(data,index):
    df_iter= data.iterrows()
    
    for index,document in df_iter:
        yield{
            "_index": 'index_k1',
            "_type": "_doc",
            "_id" : f"{index}",
            "_source": document.to_json()
        }
    raise StopIteration

In [11]:
for success,info in tqdm(helpers.parallel_bulk(es,doc_generator(df_ques[['pid','passage']],index))):
    if not success:
        print('A document failed:', info)
es.indices.refresh(index=index)

497308it [01:25, 5849.52it/s]


{'_shards': {'total': 2, 'successful': 1, 'failed': 0}}

In [12]:
def SEARCH(text,index,field):
    
    res= es.search(index=index,body={"query":{"match":{field:{"query":text,"operator":"or","fuzziness": "0"
                                                            }}}},size = 10)
    
    return([(x.get('_source'),x.get('_score')) for x in res['hits']['hits']])

In [13]:
index='index_k1'
field = 'passage'

In [14]:
#SEARCH('corporation',index,field)
question_un = list(df_ques['query'].unique())

df_new = pd.DataFrame()
df_new['question'] = question_un

In [15]:
len(question_un)

500

In [16]:
def query_generator(df):
    while True:
        yield (df_new['question'])

In [17]:
query_generator(df_new[['question']])

<generator object query_generator at 0x117dd8d00>

In [18]:
import concurrent.futures

In [19]:
%%time

queries=[]
with concurrent.futures.ThreadPoolExecutor() as executor:
    queries={executor.submit(SEARCH, query,index,'passage'): query for query in tqdm(df_new.question)}


100%|██████████| 500/500 [00:00<00:00, 2865.18it/s]


CPU times: user 595 ms, sys: 153 ms, total: 748 ms
Wall time: 5.22 s


In [20]:
out = [x.result() for x in tqdm(queries)]

100%|██████████| 500/500 [00:00<00:00, 276268.21it/s]


In [21]:
len(out)

500

In [22]:
def get_std_ref_match(data_current):
    
    #UPC to match
    questions = []
    
    #Original
    answers = []
    
    #Score
    score_bm25 = []
    
    #qid
    #qs_id = []
    
    #pid
    ps_id = []
    
        
    
    #Reindex
    data_current.index = range(0,len(data_current))
    
    
    for i in tqdm(range(0,len(data_current))):

            #indexing
            #data_current.index = range(0,len(data_current))
            
            #question
            qs = data_current['question'].iloc[i]
            
           
            #Getting the search result
            temp_out = out[i]
            val = len(temp_out)

            if val ==0:
                
                #questions
                questions.append(qs)
                

                #answer
                answers.append('NA')
                
                #qs_id ps_9d
                #qs_id.append('NA')
                ps_id.append('NA')
                
                #Score
                score_bm25.append('NA')
                
             
            else:
                for j in np.arange(val):

                    #clean_ingred
                    questions.append(qs)
                

                    #Creating temp
                    temp2 = temp_out[j]

                    #Separting the tuple
                    doc_txt = temp2[0]['passage']
                    
                    #qs_i = temp2[0]['qid']
                    ps_i = temp2[0]['pid']
                    
                    #query,passage ids
                    #qs_id.append(qs_i)
                    ps_id.append(ps_i)

                    #bm25
                    doc_scr = temp2[1]


                    #matched answers
                    answers.append(doc_txt)
                    

                    #Score
                    score_bm25.append(doc_scr)
                    
                    
                    
    

    #Converting to a Dataframe   
    df_out = pd.DataFrame()
    #df_result['qid'] = qs_id
    df_out['pid'] = ps_id
    df_out['questions'] = questions
    df_out['passage'] = answers
    df_out['score'] = score_bm25

    return df_out



In [23]:
df_out = get_std_ref_match(df_new)

100%|██████████| 500/500 [00:00<00:00, 20890.88it/s]


In [24]:
df_ques_id = df_ques[['qid','query']]

df_ques_id = df_ques_id.drop_duplicates()

In [25]:
df_out1 = pd.merge(left= df_out ,
                     right= df_ques_id,
                     left_on= 'questions',
                     right_on = 'query')

In [26]:
df_out1 = df_out1.drop(['query'],axis=1)

In [27]:
df_pred = df_out1.sort_values(by=['questions','score'],ascending = False)

In [28]:
df_pred['indices'] = range(0,len(df_pred))
df_pred['rank'] = (df_pred.groupby('questions')['indices'].rank(ascending=True))
df_pred = df_pred.drop(['indices'],axis=1)

In [29]:
df_final = df_pred.groupby('questions').head(10).reset_index(drop=True)

In [30]:
df_final_v1 = df_final[['qid','pid','rank']]


In [34]:
MaxMRRRank = 10
def compute_metrics(qids_to_relevant_passageids, qids_to_ranked_candidate_passages):
    """Compute MRR metric
    Args:    
    p_qids_to_relevant_passageids (dict): dictionary of query-passage mapping
        Dict as read in with load_reference or load_reference_from_stream
    p_qids_to_ranked_candidate_passages (dict): dictionary of query-passage candidates
    Returns:
        dict: dictionary of metrics {'MRR': <MRR Score>}
    """
    all_scores = {}
    MRR = 0
    qids_with_relevant_passages = 0
    ranking = []
    list1=list(set(qids_to_ranked_candidate_passages['qid'].to_list()))
    #print(list1)
    list2=qids_to_relevant_passageids['qid'].to_list()
    for qid in list1:
        if qid in list2:
            ranking.append(0)
            target_pid = list(qids_to_relevant_passageids[qids_to_relevant_passageids['qid']==qid]['pid'])
            candidate_pid = list(qids_to_ranked_candidate_passages[qids_to_ranked_candidate_passages['qid']==qid]['pid'])
            for i in range(0,MaxMRRRank):
                if candidate_pid[i] in target_pid:
                    MRR += 1/(i + 1)
                    #print(MRR)
#                     ranking.pop()
#                     ranking.append(i+1)
                    break
#     if len(ranking) == 0:
#         raise IOError("No matching QIDs found. Are you sure you are scoring the evaluation set?")
# #     print(MRR)
#     print(len(qids_to_relevant_passageids))
    MRR = MRR/len(list1)
    all_scores['MRR @10'] = MRR
    all_scores['QueriesRanked'] = qids_to_ranked_candidate_passages['qid'].nunique()
    return all_scores

In [35]:
mrr = compute_metrics(df_res, df_final_v1)

In [36]:
mrr

{'MRR @10': 0.13552063492063499, 'QueriesRanked': 500}

In [ ]:
#df_final_v1['qid'].unique()[0:20]

In [ ]:
#df_final_v1[df_final_v1['qid'] == 1066792]

#### Evaluation

In [39]:
# MAX_RANK = 10

# hits = pd.merge(df_res, df_final_v1,
#     on=["qid", "pid"],
#     how="left").fillna(MAX_RANK)

# mrr = (1 / hits.groupby('qid')['rank'].min()).mean()